In [62]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/employee-attrition-and-engagement/employee_attrition_and_engagement.csv


In [173]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score, confusion_matrix
from sklearn import preprocessing
preprocessing.LabelEncoder()
import random
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

In [64]:
df=pd.read_csv('/kaggle/input/employee-attrition-and-engagement/employee_attrition_and_engagement.csv', header=[0])
df.head()

,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,...,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,1102,Sales,1,2,Life Sciences,1,1,...,1,80,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,279,Research & Development,8,1,Life Sciences,1,2,...,4,80,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,1373,Research & Development,2,2,Other,1,4,...,2,80,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,1392,Research & Development,3,4,Life Sciences,1,5,...,3,80,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,591,Research & Development,2,1,Medical,1,7,...,4,80,1,6,3,3,2,2,2,2


In [65]:
df.shape

(1470, 35)

In [66]:
df.isnull().sum().sum()

0

In [67]:
cols = ["Attrition", "BusinessTravel","Department", "EducationField","Gender", "JobRole","MaritalStatus", "Over18","OverTime"]

In [68]:
le=preprocessing.LabelEncoder()
for i in cols:
    df[i]=le.fit_transform(list(df[i]))

In [69]:
df.head()

,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,...,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,1,2,1102,2,1,2,1,1,1,...,1,80,0,8,0,1,6,4,0,5
1,49,0,1,279,1,8,1,1,1,2,...,4,80,1,10,3,3,10,7,1,7
2,37,1,2,1373,1,2,2,4,1,4,...,2,80,0,7,3,3,0,0,0,0
3,33,0,1,1392,1,3,4,1,1,5,...,3,80,0,8,3,3,8,7,3,0
4,27,0,2,591,1,2,1,3,1,7,...,4,80,1,6,3,3,2,2,2,2


In [70]:
y= df['Attrition']
y=y.astype('int')

In [71]:
x = df.drop('Attrition', axis=1)

In [72]:
x.head()

,Age,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,EnvironmentSatisfaction,...,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,2,1102,2,1,2,1,1,1,2,...,1,80,0,8,0,1,6,4,0,5
1,49,1,279,1,8,1,1,1,2,3,...,4,80,1,10,3,3,10,7,1,7
2,37,2,1373,1,2,2,4,1,4,4,...,2,80,0,7,3,3,0,0,0,0
3,33,1,1392,1,3,4,1,1,5,4,...,3,80,0,8,3,3,8,7,3,0
4,27,2,591,1,2,1,3,1,7,1,...,4,80,1,6,3,3,2,2,2,2


In [73]:
def calaculate_accuracy(x):
    X=df[x]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.30)
    lgt_R = LogisticRegression(solver='lbfgs')
    lgt_R.fit(X_train, y_train)
    accuracy=lgt_R.score(X_test,y_test)
    print('Accuracy:',accuracy)
    return accuracy

In [74]:
columnsName=df.drop(labels= 'Attrition', axis= 1).columns.values.tolist()

In [75]:
print(columnsName)

['Age', 'BusinessTravel', 'DailyRate', 'Department', 'DistanceFromHome', 'Education', 'EducationField', 'EmployeeCount', 'EmployeeNumber', 'EnvironmentSatisfaction', 'Gender', 'HourlyRate', 'JobInvolvement', 'JobLevel', 'JobRole', 'JobSatisfaction', 'MaritalStatus', 'MonthlyIncome', 'MonthlyRate', 'NumCompaniesWorked', 'Over18', 'OverTime', 'PercentSalaryHike', 'PerformanceRating', 'RelationshipSatisfaction', 'StandardHours', 'StockOptionLevel', 'TotalWorkingYears', 'TrainingTimesLastYear', 'WorkLifeBalance', 'YearsAtCompany', 'YearsInCurrentRole', 'YearsSinceLastPromotion', 'YearsWithCurrManager']


In [76]:
columnsName1=[0,1]
chromosomes=[]
for i in range(10):
    chro1=[]
    for i in range(34):
        item = random.choice(tuple(columnsName1))
        chro1.append(item)
    chromosomes.append(chro1)

In [77]:
def data(chromosomes1):
    chromosomes2=[]
    for i in range(len(chromosomes1)):
        if chromosomes1[i]==1:
                chromosomes2.append(columnsName[i])
    return chromosomes2

In [78]:
pb=[]
def checkpersonalbest():
    for i in range(len(chromosomes)):
         pb.append(calaculate_accuracy(data(chromosomes[i])))
checkpersonalbest()

Accuracy: 0.8321995464852607
Accuracy: 0.854875283446712
Accuracy: 0.8707482993197279
Accuracy: 0.8458049886621315
Accuracy: 0.8208616780045351
Accuracy: 0.8571428571428571
Accuracy: 0.8367346938775511
Accuracy: 0.8140589569160998
Accuracy: 0.8412698412698413
Accuracy: 0.8231292517006803


In [79]:
def checkvelocity(globalbest):
    velocity=[]
    for j in range(len(chromosomes)):
        velocity.append(list(0+1*(np.random.random(1)[0])*(np.array(chromosomes[j])-np.array(chromosomes[j]))+1*(np.random.random(1)[0])*(np.array(globalbest)-np.array(chromosomes[j]))))
    #print(velocity)
    return velocity

In [80]:
def addingchromosomes(velocity):
    chromosomes2=[]
    for i in range(len(velocity)):
        nextchromo=[]
        for j in range(len(velocity[i])):
            nextchromo.append(chromosomes[i][j]+velocity[i][j])
        chromosomes2.append(nextchromo)
    return chromosomes2

In [81]:
def normalize(chromosomes2):
    for l in range(len(chromosomes2)):
        for m in range(len(chromosomes2[l])):
            if chromosomes2[l][m]>0.5:
                chromosomes2[l][m]=1
            else:
                chromosomes2[l][m]=0
    return chromosomes2

In [82]:
def checkpd(chromosomes2):
    personal=[]
    for i in range(len(chromosomes2)):
        personal.append(calaculate_accuracy(data(chromosomes2[i])))
    for j in range(len(personal)):
        if(personal[j]>pb[j]):
            chromosomes[j]=chromosomes2[j]
            pb[j]=personal[j]
    return personal

In [83]:
max(pb)
ind = pb.index(max(pb))
globalbest=chromosomes[ind]
for i in range(20):
    chromosomes2=[]
    personal=[]
    velocity=checkvelocity(globalbest)
    chromosomes2=addingchromosomes(velocity)
    chromosomes2=normalize(chromosomes2)
    personal=checkpd(chromosomes2)
    globalbest=[]
    max(pb)
    ind = pb.index(max(pb))
    globalbest=chromosomes[ind]

Accuracy: 0.8662131519274376
Accuracy: 0.8344671201814059
Accuracy: 0.8458049886621315
Accuracy: 0.8321995464852607
Accuracy: 0.8095238095238095
Accuracy: 0.8458049886621315
Accuracy: 0.81859410430839
Accuracy: 0.8231292517006803
Accuracy: 0.8458049886621315
Accuracy: 0.8367346938775511
Accuracy: 0.8571428571428571
Accuracy: 0.8321995464852607
Accuracy: 0.8526077097505669
Accuracy: 0.854875283446712
Accuracy: 0.8412698412698413
Accuracy: 0.8480725623582767
Accuracy: 0.8503401360544217
Accuracy: 0.8208616780045351
Accuracy: 0.8390022675736961
Accuracy: 0.8299319727891157
Accuracy: 0.8503401360544217
Accuracy: 0.8526077097505669
Accuracy: 0.8253968253968254
Accuracy: 0.8140589569160998
Accuracy: 0.8321995464852607
Accuracy: 0.8276643990929705
Accuracy: 0.8503401360544217
Accuracy: 0.8344671201814059
Accuracy: 0.8480725623582767
Accuracy: 0.8594104308390023
Accuracy: 0.8458049886621315
Accuracy: 0.8571428571428571
Accuracy: 0.8299319727891157
Accuracy: 0.8458049886621315
Accuracy: 0.85941

In [84]:
max(pb)

0.8866213151927438

In [85]:
ind = pb.index(max(pb))
print(ind)
globalbest=chromosomes[ind]

1


In [86]:
print(data(globalbest))

['DailyRate', 'DistanceFromHome', 'Education', 'EmployeeCount', 'EnvironmentSatisfaction', 'JobRole', 'JobSatisfaction', 'MaritalStatus', 'MonthlyIncome', 'NumCompaniesWorked', 'OverTime', 'PercentSalaryHike', 'RelationshipSatisfaction', 'StandardHours', 'WorkLifeBalance', 'YearsAtCompany', 'YearsSinceLastPromotion']


In [185]:
# x2 = df[data(globalbest)].copy()
x2 = df[data(globalbest)]

In [186]:
x2.head()

,DailyRate,DistanceFromHome,Education,EmployeeCount,EnvironmentSatisfaction,JobRole,JobSatisfaction,MaritalStatus,MonthlyIncome,NumCompaniesWorked,OverTime,PercentSalaryHike,RelationshipSatisfaction,StandardHours,WorkLifeBalance,YearsAtCompany,YearsSinceLastPromotion
0,1102,1,2,1,2,7,4,2,5993,8,1,11,1,80,1,6,0
1,279,8,1,1,3,6,2,1,5130,1,0,23,4,80,3,10,1
2,1373,2,2,1,4,2,3,2,2090,6,1,15,2,80,3,0,0
3,1392,3,4,1,4,6,3,1,2909,1,1,11,3,80,3,8,3
4,591,2,1,1,1,2,2,1,3468,9,0,12,4,80,3,2,2


In [164]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size= 0.30)
lgt_R = LogisticRegression(solver='lbfgs')
lgt_R.fit(X_train, y_train)
accuracy=lgt_R.score(X_test,y_test)
    
y_pred = lgt_R.predict(X_test)
# knn = KNeighborsClassifier()
# knn.fit(X_train, y_train)

# y_pred = knn.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
print('Accuracy:',accuracy)
print('Precision:', precision)
print('Recall:', recall)
print('F1-score:', f1)

Accuracy: 0.8435374149659864
Precision: 0.375
Recall: 0.04477611940298507
F1-score: 0.08


In [109]:
conf_matrix = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(conf_matrix)

Confusion Matrix:
[[352  17]
 [ 67   5]]


In [223]:
def calculate_metrics(x, penalty='l2', solver='lbfgs'):
    X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.30)
    
    lgt_R = LogisticRegression(penalty=penalty, solver=solver)
    lgt_R.fit(X_train, y_train)
    
    y_pred = lgt_R.predict(X_test)
    
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    conf_matrix = confusion_matrix(y_test, y_pred)
    
    print('Accuracy:', accuracy)
    print('Precision:', precision)
    print('Recall:', recall)
    print('F1-score:', f1)
    print('Confusion Matrix:', conf_matrix)

    
    return accuracy, precision, recall, f1, conf_matrix

In [226]:
calculate_metrics(x2)

param_grid = {
    'penalty': ['l1', 'l2'],
    'solver': ['liblinear', 'lbfgs', 'sag', 'saga']
}

grid_search = GridSearchCV(LogisticRegression(), param_grid, cv=5)
grid_search.fit(X_train, y_train)

best_params = grid_search.best_params_
print("Best hyperparameters:", best_params)

calculate_metrics(x2, **best_params)

Accuracy: 0.8435374149659864
Precision: 0.0
Recall: 0.0
F1-score: 0.0
Confusion Matrix: [[372   2]
 [ 67   0]]
Best hyperparameters: {'penalty': 'l1', 'solver': 'liblinear'}
Accuracy: 0.8662131519274376
Precision: 0.6666666666666666
Recall: 0.2898550724637681
F1-score: 0.40404040404040403
Confusion Matrix: [[362  10]
 [ 49  20]]


(0.8662131519274376,
 0.6666666666666666,
 0.2898550724637681,
 0.40404040404040403,
 array([[362,  10],
        [ 49,  20]]))